In [4]:
import numpy as np
import pandas as pd
from multiprocessing import Pool
import matplotlib.pyplot as plt
import matplotlib.gridspec as grid_spec
from tqdm.auto import tqdm
from math import ceil
import itertools
import h5py
import io
import robustbench

In [5]:
dataset_path = "/data/output/20220226_robustness/dataset.h5"
df_meta = pd.read_hdf(dataset_path, "meta")
df_meta["filter_ids"] = df_meta["filter_ids"].apply(lambda s: np.arange(int(s.split(":")[0]), 1 + int(s.split(":")[1])))
df_meta.Robust = df_meta.Robust.apply(bool)

#### Number of models

In [6]:
df_meta.index.get_level_values(0).unique().size

119

In [7]:
df_meta[df_meta.Robust].index.get_level_values(0).unique().size

73

In [8]:
df_meta[~df_meta.Robust].index.get_level_values(0).unique().size

46

In [9]:
df_meta.columns

Index(['filter_ids', 'model', 'path', 'producer', 'op_set', 'Conv',
       '(3, 3) filters', 'BatchNormalization', 'Add', 'Relu', '(1, 1) filters',
       'Pad', 'AveragePool', 'Reshape', 'Gemm', 'depth', 'Sub', 'Div',
       'Sigmoid', 'Mul', 'Shape', 'Gather', 'Unsqueeze', 'Concat',
       'ReduceMean', 'ReduceProd', 'Cast', 'Sqrt', 'Max', '(7, 7) filters',
       'MaxPool', 'GlobalAveragePool', 'Flatten', 'Softmax', 'ReduceL2',
       'Clip', 'MatMul', 'Expand', 'Name', 'Paper', 'Tracer Warning',
       'Selection', 'Hide', 'Robust', 'Clean Accuracy', 'Robust Accuracy',
       'Network', 'Backbone', 'Framework', 'Pretraining-Dataset',
       'Training-Dataset', 'Visual Category', 'Visual Category_micro',
       'Precision', 'Conv Weight Initializer', 'Optimizer', 'Task',
       'Input Shape', 'Accessible', 'total_filters', '3x3_filter_share'],
      dtype='object')

#### Number of depth (NOT layers!)

In [6]:
df_meta.index.size

3667

In [7]:
df_meta.groupby(["Robust", "Training-Dataset", "Backbone",]).Name.nunique()

Robust  Training-Dataset  Backbone               
False   cifar10           PreActResNet-18             3
                          ResNet-18                   2
                          ResNet-50                   2
                          WideResNet-106-16           1
                          WideResNet-28-10            3
                          WideResNet-28-10-PSSiLU     1
                          WideResNet-28-4             1
                          WideResNet-34-10            7
                          WideResNet-34-15            1
                          WideResNet-34-20            5
                          WideResNet-34-R             1
                          WideResNet-70-16            2
        cifar100          PreActResNet-18             3
                          WideResNet-28-10            2
                          WideResNet-34-10            4
                          WideResNet-34-20            1
                          WideResNet-70-16            

In [8]:
df_meta.groupby(["Robust", "Training-Dataset"]).Name.nunique()

Robust  Training-Dataset
False   cifar10             29
        cifar100            11
        imagenet             6
True    cifar10             49
        cifar100            17
        imagenet             5
Name: Name, dtype: int64

In [10]:
from robustbench.model_zoo import model_dicts as all_models
from robustbench.model_zoo.enums import BenchmarkDataset, ThreatModel
import inspect
from collections import defaultdict

paper_dict = defaultdict(lambda: defaultdict(list))
for dataset in BenchmarkDataset:
    for paper, info_dict in all_models[dataset][ThreatModel.Linf].items():
        paper_dict[dataset.value][inspect.getsource(info_dict["model"])].append(paper)

In [11]:
for dataset, models in paper_dict.items():
    for pair in models.values():
        print(dataset, pair, df_meta[(df_meta.Network.isin(pair)) & (df_meta["Training-Dataset"] == dataset)].Backbone.unique())

cifar10 ['Andriushchenko2020Understanding'] ['PreActResNet-18']
cifar10 ['Carmon2019Unlabeled', 'Sehwag2020Hydra', 'Wang2020Improving', 'Wu2020Adversarial_extra', 'Zhang2020Geometry', 'Sridhar2021Robust'] ['WideResNet-28-10']
cifar10 ['Hendrycks2019Using'] ['WideResNet-28-10']
cifar10 ['Rice2020Overfitting'] ['WideResNet-34-20']
cifar10 ['Zhang2019Theoretically', 'Huang2020Self', 'Zhang2019You', 'Zhang2020Attacks', 'Sitawarin2020Improving', 'Cui2020Learnable_34_10'] ['WideResNet-34-10']
cifar10 ['Engstrom2019Robustness'] ['ResNet-50']
cifar10 ['Chen2020Adversarial'] ['ResNet-50']
cifar10 ['Pang2020Boosting'] ['WideResNet-34-20']
cifar10 ['Wong2020Fast'] ['PreActResNet-18']
cifar10 ['Ding2020MMA'] ['WideResNet-28-4']
cifar10 ['Wu2020Adversarial'] ['WideResNet-34-10']
cifar10 ['Gowal2020Uncovering_70_16', 'Gowal2020Uncovering_70_16_extra', 'Rebuffi2021Fixing_70_16_cutmix_ddpm', 'Rebuffi2021Fixing_70_16_cutmix_extra'] ['WideResNet-70-16']
cifar10 ['Gowal2020Uncovering_34_20'] ['WideResNet

In [12]:
for dataset in BenchmarkDataset:
    for paper, info_dict in all_models[dataset][ThreatModel.Linf].items():
        if not paper in df_meta.Network.values:
            print(paper, dataset, "is missing")

Standard BenchmarkDataset.cifar_10 is missing
Kang2021Stable BenchmarkDataset.cifar_10 is missing
Standard_R50 BenchmarkDataset.imagenet is missing


In [13]:
paper_dict["imagenet"].values()

dict_values([['Wong2020Fast', 'Engstrom2019Robustness', 'Salman2020Do_R50'], ['Salman2020Do_R18'], ['Salman2020Do_50_2'], ['Standard_R50']])

In [16]:
pairings = []
for dataset in BenchmarkDataset:
    for paper, info_dict in all_models[dataset][ThreatModel.Linf].items():
        robust_model = df_meta[(df_meta.Network == paper) & (df_meta["Training-Dataset"] == dataset.value) & (df_meta.Robust)].Name.unique()
        
        if len(robust_model) < 1:
            print(paper, dataset.value)
            continue
        
        for pairs in paper_dict[dataset.value].values():
            if paper in pairs:
                counter_parts = pairs
        
        normal_model = df_meta[(df_meta.Network.isin(counter_parts)) & (df_meta["Training-Dataset"] == dataset.value) & (~df_meta.Robust)].Name.unique()
        backbone = df_meta[(df_meta.Network.isin(counter_parts)) & (df_meta["Training-Dataset"] == dataset.value)].Backbone.unique()
        pairings.append((robust_model[0], normal_model[0], backbone[0]))
        
df = pd.DataFrame(pairings, columns=["Robust", "Normal", "Arch"])
df.to_csv("robust_vs_normal_pairings.csv")
df

Standard cifar10
Kang2021Stable cifar10
Standard_R50 imagenet


,Robust,Normal,Arch
0,robustbench_Andriushchenko2020Understanding_ci...,hso_normal_training_robustbench_Andriushchenko...,PreActResNet-18
1,robustbench_Carmon2019Unlabeled_cifar10_linf_13,hso_normal_training_robustbench_Carmon2019Unla...,WideResNet-28-10
2,robustbench_Sehwag2020Hydra_cifar10_linf_13,hso_normal_training_robustbench_Carmon2019Unla...,WideResNet-28-10
3,robustbench_Wang2020Improving_cifar10_linf_13,hso_normal_training_robustbench_Carmon2019Unla...,WideResNet-28-10
4,robustbench_Hendrycks2019Using_cifar10_linf_13,hso_normal_training_robustbench_Hendrycks2019U...,WideResNet-28-10
...,...,...,...
66,robustbench_Wong2020Fast_imagenet_linf_13,hso_normal_training_robustbench_Wong2020Fast_i...,ResNet-50
67,robustbench_Engstrom2019Robustness_imagenet_li...,hso_normal_training_robustbench_Wong2020Fast_i...,ResNet-50
68,robustbench_Salman2020Do_R50_imagenet_linf_13,hso_normal_training_robustbench_Wong2020Fast_i...,ResNet-50
69,robustbench_Salman2020Do_R18_imagenet_linf_13,hso_normal_training_robustbench_Salman2020Do_R...,ResNet-18
